# Creating Column Indexes

This is a short tutorial for creating indexes on table columns. In general, DESDM uses bitmap indexes, though in some cases b-tree indexes are used. One cool feature of Oracle's bitmap indexing is that individual indexes can be combined. For example, indexes on ```spread_model``` and ```spreaderr_model``` can lead to a performance increase for queries on the quantity ```spread_model + 3*spreaderr_model```. More details on indexing can be found [here](http://www.oracle.com/technetwork/articles/sharma-indexes-093638.html).

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import easyaccess as ea

In [3]:
size = (100)
x = np.random.normal(size=size)
y = np.random.randint(0,size,size)
z = y.astype(str)
data = np.rec.fromarrays([x,y,z],names=['xcol','ycol','zcol'])

In [4]:
conn = ea.connect(section='dessci',quiet=True)

In [5]:
table_idx = 'mytable_index'
query = """
--- Custom create table query
CREATE TABLE %s (
xcol NUMBER(6,4) NOT NULL, 
ycol BINARY_DOUBLE, 
zcol VARCHAR2(5) )"""%table_idx
conn.cursor().execute(query)

In [6]:
columns = data.dtype.names
values = data.tolist()
conn.insert_data(columns,values,table_idx)


 Inserted 100 rows and 3 columns into table MYTABLE_INDEX in 0.05 seconds


In [7]:
idxcol = 'xcol'
idxname = 'xidx'
query = 'CREATE BITMAP INDEX %s on %s(%s)'%(idxname,table_idx,idxcol)
conn.cursor().execute(query)
conn.onecmd('show_index %s'%table_idx)



1 rows in 0.09 seconds

      TABLE_NAME COLUMN_NAME INDEX_TYPE INDEX_NAME ITYP_NAME
1  MYTABLE_INDEX        XCOL     BITMAP       XIDX      None



Indexes can also be created as a "primary key" field.

In [8]:
i = np.arange(size)
data = np.rec.fromarrays([i,x,y,z],names=['key','xcol','ycol','zcol'])
keycol = 'key'
keyname = 'key_pk'
table_key = 'mytable_key'
query = """
--- Create table with primary key
CREATE TABLE %s (
key NUMBER(6,0) NOT NULL,
xcol NUMBER(6,4) NOT NULL, 
ycol BINARY_DOUBLE, 
zcol VARCHAR2(5), 
CONSTRAINT %s PRIMARY KEY (%s) ) """%(table_key,keyname,keycol)
conn.cursor().execute(query)

In [9]:
print data['key']

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99]


In [10]:
columns = data.dtype.names
values = data.tolist()
conn.insert_data(columns,values,table_key)


 Inserted 100 rows and 4 columns into table MYTABLE_KEY in 0.05 seconds


In [11]:
conn.onecmd('show_index %s'%table_key)



1 rows in 0.08 seconds

    TABLE_NAME COLUMN_NAME INDEX_TYPE INDEX_NAME ITYP_NAME
1  MYTABLE_KEY         KEY     NORMAL     KEY_PK      None



In [12]:
conn.cursor().execute('DROP TABLE %s PURGE'%table_idx)
conn.cursor().execute('DROP TABLE %s PURGE'%table_key)
conn.close()